In [2]:
%matplotlib widget
import numpy as np
from peakpy.optical import fit_gaussian as fg
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy.optimize import fsolve
import os
from importlib import reload
from common import load_angle_vectors, spherical_to_cartesian, find_z_rotation_matrix,\
                    plot_cone_surfaces, cone_surface, find_intersections, str2mc

In [45]:
msr_label = 'Pre-VibeBCRS'
angles = load_angle_vectors('measurements/' + msr_label + '.json', which = 'b')
strVecTh2_strB = spherical_to_cartesian(angles["thSTR_mSTR_str"])
bcVecTh1_strB = spherical_to_cartesian(angles["thBC_mBC_str"])
th2RefVec_strB = spherical_to_cartesian(angles["thSTR_mTH_str"])
th1RefVec_strB = spherical_to_cartesian(angles["thBC_mTH_str"])

strVecTh2_ssB = spherical_to_cartesian(angles["thSTR_mSTR_ss"])
bcVecTh1_ssB = spherical_to_cartesian(angles["thBC_mBC_ss"])
th2RefVec_ssB = spherical_to_cartesian(angles["thSTR_mTH_ss"])
th1RefVec_ssB = spherical_to_cartesian(angles["thBC_mTH_ss"])

In [46]:
angles = load_angle_vectors('measurements/' + msr_label + '.json', which = 'a')

strVecTh2_strA = spherical_to_cartesian(angles["thSTR_mSTR_str"])
bcVecTh1_strA = spherical_to_cartesian(angles["thBC_mBC_str"])
th2RefVec_strA = spherical_to_cartesian(angles["thSTR_mTH_str"])
th1RefVec_strA = spherical_to_cartesian(angles["thBC_mTH_str"])

strVecTh2_ssA = spherical_to_cartesian(angles["thSTR_mSTR_ss"])
bcVecTh1_ssA = spherical_to_cartesian(angles["thBC_mBC_ss"])
th2RefVec_ssA = spherical_to_cartesian(angles["thSTR_mTH_ss"])
th1RefVec_ssA = spherical_to_cartesian(angles["thBC_mTH_ss"])


In [47]:
strVecTh2_str = (strVecTh2_strA + strVecTh2_strB) / 2
bcVecTh1_str = (bcVecTh1_strA + bcVecTh1_strB) / 2
th2RefVec_str = (th2RefVec_strA + th2RefVec_strB) / 2
th1RefVec_str = (th1RefVec_strA + th1RefVec_strB) / 2

strVecTh2_ss = (strVecTh2_ssA + strVecTh2_ssB) / 2
bcVecTh1_ss = (bcVecTh1_ssA + bcVecTh1_ssB) / 2
th2RefVec_ss = (th2RefVec_ssA + th2RefVec_ssB) / 2
th1RefVec_ss = (th1RefVec_ssA + th1RefVec_ssB) / 2

vect_uncs = {
    "strVecTh2_str": np.linalg.norm(strVecTh2_strA - strVecTh2_strB) / 2,
    "bcVecTh1_str": np.linalg.norm(bcVecTh1_strA - bcVecTh1_strB) / 2,
    "th2RefVec_str": np.linalg.norm(th2RefVec_strA - th2RefVec_strB) / 2,
    "th1RefVec_str": np.linalg.norm(th1RefVec_strA - th1RefVec_strB) / 2,
    "strVecTh2_ss": np.linalg.norm(strVecTh2_ssA - strVecTh2_ssB) / 2,
    "bcVecTh1_ss": np.linalg.norm(bcVecTh1_ssA - bcVecTh1_ssB) / 2,
    "th2RefVec_ss": np.linalg.norm(th2RefVec_ssA - th2RefVec_ssB) / 2,
    "th1RefVec_ss": np.linalg.norm(th1RefVec_ssA - th1RefVec_ssB) / 2
}

In [48]:
tot_unc = 0
for vv in vect_uncs.keys():
    print(vv)
    msr_unc = vect_uncs[vv]*265265
    print(msr_unc)
    tot_unc += msr_unc

strVecTh2_str
4.861235022098153
bcVecTh1_str
2.5568614224427364
th2RefVec_str
19.839146612660812
th1RefVec_str
1.4640540197709129
strVecTh2_ss
1.636826822244396
bcVecTh1_ss
18.572405642623572
th2RefVec_ss
8.115270336835668
th1RefVec_ss
2.096202491308841


In [49]:
tot_unc

59.14200236998509

In [50]:
str_rot = find_z_rotation_matrix(th1RefVec_str,bcVecTh1_str )
ss_rot = find_z_rotation_matrix(th1RefVec_ss,bcVecTh1_ss )

63.511449999999165
64.24235000057604


In [51]:
bcVecTh2_str = - str_rot @ th2RefVec_str
bcVecTh2_ss = - ss_rot @ th2RefVec_ss

print(bcVecTh2_str)
print(bcVecTh2_ss)

[-0.99975194 -0.02204248 -0.00319133]
[-0.99941493 -0.02799038  0.0196555 ]


In [52]:
print('Relative angle with STR prism side is: ')
angle_str = np.arccos(np.sum(bcVecTh2_str* strVecTh2_str)) * 180 / np.pi
print(angle_str)

print('Relative angle with SS prism side is: ')
angle_ss = np.arccos(np.sum(bcVecTh2_ss* strVecTh2_ss)) * 180 / np.pi
print(angle_ss)

print('If you want')

Relative angle with STR prism side is: 
90.13242996972018
Relative angle with SS prism side is: 
87.79222014674282
If you want


In [53]:
plot_cone_surfaces(angle_str = angle_str, angle_ss=angle_ss)

0.9999999999999999
[-0.038523492703680405, -0.9992550216193312, -0.0023113369419610306]


In [54]:
intersections = find_intersections(angle_str, angle_ss)

0.9999999999999999


In [55]:
bcPrism_vec = intersections[0]
print('The final vector is:')
print(intersections[0])
print('The Norm is : ', np.linalg.norm(bcPrism_vec))

print('The X missalignment is')
print(180 / np.pi * np.arcsin(intersections[0][0]))
print('The Y missalignment is')
print(180 / np.pi * np.arcsin(intersections[0][1]))
print('The total misalignment is:')
print(180/np.pi * np.arccos(intersections[0][-1]))

The final vector is:
[-0.038523492703680405, -0.9992550216193312, -0.0023113369419610306]
The Norm is :  0.9999999999999999
The X missalignment is
-2.2077798532571777
The Y missalignment is
-87.78824799099847
The total misalignment is:
90.13242996972019


In [56]:
bcStr_vec = str2mc.T @ bcPrism_vec
print('The final vector is:')
print(bcStr_vec)
print('The Norm is : ', np.linalg.norm(bcStr_vec))
print('The X missalignment is')
print(180 / np.pi * np.arcsin(bcStr_vec[0]))
print('The Y missalignment is')
print(180 / np.pi * np.arcsin(bcStr_vec[1]))
print('The total misalignment is:')
print(180/np.pi * np.arccos(bcStr_vec[-1]))

The final vector is:
[-0.04476051 -0.99896492 -0.00809874]
The Norm is :  1.000000005832728
The X missalignment is
-2.565445283209317
The Y missalignment is
-87.39287859297147
The total misalignment is:
90.46402888667802


In [57]:
from peakpy.transformations.rotation_matrices import Rxyz
str2sbc = Rxyz(90, 0, -2.5).T
bcSbc_vec = str2sbc @ bcStr_vec
print('The final vector is:')
print(bcSbc_vec)
print('The Norm is : ', np.linalg.norm(bcSbc_vec))

print('The X missalignment is')
print(180 / np.pi * np.arcsin(bcSbc_vec[0]))
print('The Y missalignment is')
print(180 / np.pi * np.arcsin(bcSbc_vec[1]))
print('The total misalignment is:')
print(180/np.pi * np.arccos(bcSbc_vec[-1]))

The final vector is:
[-0.00114313 -0.00809874  0.99996836]
The Norm is :  1.0000018091628249
The X missalignment is
-0.06549680913117652
The Y missalignment is
-0.4640288866780114
The total misalignment is:
0.45577849736479925


In [44]:
import json
with open('calcVecs.json', 'r') as f:
    calc_vecs = json.load(f)


calc_vecs[msr_label] = list(bcSbc_vec)
print('Vector Saved Succesfully')
with open('calcVecs.json', 'w') as f:
    json.dump(calc_vecs, f, indent=4)
# if msr_label not in calc_vecs.keys():


# else:
#     print('Vector with this label already exists. Are you sure what you are doing is correct?')

Vector Saved Succesfully
